# Train/Test

Perform simple experiments training a naive and an XgBoost classifiers to classify Urban Atlas land-use categories from Foursquare POIs.

### Setup

In [1]:
%matplotlib inline

import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point, Polygon
import os
import re
import zipfile,fnmatch
import matplotlib.pyplot as plt
import pathlib
import sys

import xgboost
from sklearn.svm import SVR

from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import learning_curve
from sklearn.metrics import mean_squared_error
from sklearn.svm import LinearSVR
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics


In [1]:
# SET CITY NAME
CITIES= ["barcelona", "london","paris","milan","amsterdam","lisbon"]
CITY_NAME= CITIES[0]

In [2]:
tessellation_sizes = [50,100,200,250]
SIZE = 200

In [4]:
# Base directory
BASE_DIR = os.path.abspath(".")
# base directory for data files
BASE_DIR_DATA = os.path.join(BASE_DIR, "data")
# Base dir for a city data
BASE_DIR_CITY = os.path.join(BASE_DIR_DATA , CITY_NAME)
# Base dir for the city tesselation/grid
BASE_DIR_TESSELLATION = os.path.join(BASE_DIR_CITY, "tessellation")
# Empty tesselation/grid 
TESSELLATION = os.path.join(BASE_DIR_TESSELLATION, "tessellation_"+CITY_NAME +"-Square-"+ str(SIZE) +".geojson")
# Base dir landuse from Urban Atlas
BASE_DIR_LANDUSE = os.path.join(BASE_DIR_CITY, 'landuse')
# Base dir landuse FourSquare
BASE_DIR_FOURSQUARE = os.path.join(BASE_DIR_CITY, 'foursquare_raw')
# City shape fron Nominatim API
OUTPUT_CITY_SHAPE = os.path.join(BASE_DIR_CITY,'osm_raw', CITY_NAME + ".geojson")
#  POIs in Polygon(city shape), output of the clipping operation
OUTPUT_POIS = os.path.join(BASE_DIR_CITY, CITY_NAME +"_POIs.csv")
# Foursquare raw data (formerly known as FOURSQUARE_GRID)
FOURSQUARE_RAW_DATA = os.path.join(BASE_DIR_FOURSQUARE, CITY_NAME +"_poi.csv")
# Foursquare mapped to the grid/tessellation
FOURSQUARE_TESSELLATION = os.path.join(BASE_DIR_CITY, 'mapped', CITY_NAME +"_fs_tessellation_"+ str(SIZE) +".csv")
# Count of Foursquare features per each cell in the tessellation
FOURSQUARE_COUNT = os.path.join(BASE_DIR_CITY, 'count', CITY_NAME +"_fs_count_"+ str(SIZE) +".csv")
# Count of Urban Atlas features per each cell in the tessellation
UA_COUNT = os.path.join(BASE_DIR_CITY, 'count', CITY_NAME +"_ua_count_"+ str(SIZE) +".csv")


### Merge  Urban Atlas + Foursquare POI datasets

Merging **FourSquare POI** derived from count stage onto **UA landuse** (both mapped with `poi_mapping.py`). Then divide them in Train/Test.

In [19]:
def mergeUA_POI(CITY_NAME, SIZE):
    
    # input Foursquare data mapped to grid
    FOURSQUARE_COUNT = os.path.join(BASE_DIR_CITY, 'count', CITY_NAME +"_fs_count_"+ SIZE +".csv")
    # input UA data mapped to grid
    UA_COUNT = os.path.join(BASE_DIR_CITY, 'count', CITY_NAME +"_ua_count_"+ SIZE +".csv")

    # load Foursquare Count
    foursquare_tessellation = pd.read_csv(FOURSQUARE_COUNT)
    foursquare_tessellation.columns = list(map(lambda x : 'f_fs_'+x if x != "cellID" else x, foursquare_tessellation.columns))
    # foursquare_tessellation['f_fs_Highway'] = osm_tessellation['f_osm_Highway']
    foursquare_tessellation.head(2)

    # load UA mapped onto grid
    ua_tessellation = pd.read_csv(UA_COUNT)
    ua_tessellation.columns = list(map(lambda x : 't_'+x if x != "cellID" else x, ua_tessellation.columns))


    # select only relevant columns
    ua_tessellation_target = ua_tessellation.loc[:,['cellID','t_predominant']]
    
    # Merge the UA and Foursquare dataframes
    df_ua_fs = ua_tessellation_target.merge(foursquare_tessellation, on="cellID", how='left')
#     print('SPEREM----------------------------------------------------------')
#     print(df_ua_fs[df_ua_fs['cellID']==39269])

    return df_ua_fs

###  Divide Train/Test

Split the dataset in the usual percentages of 80-20

In [20]:
def split_train_test(df_ua_fs, CITY_NAME, SIZE):

    # output UA data mapped to grid
    OUTPUT_TRAIN,OUTPUT_TEST = [os.path.join(BASE_DIR_CITY, STEP, CITY_NAME +"_fs_"+ SIZE +".csv") for STEP in ["train","test"]]
    OUTPUT_TRAIN_SCALED = '_scaled_.'.join(OUTPUT_TRAIN.split('.'))
    OUTPUT_TEST_SCALED = '_scaled_.'.join(OUTPUT_TEST.split('.'))
    print(OUTPUT_TRAIN, OUTPUT_TEST)
    print(OUTPUT_TRAIN_SCALED, OUTPUT_TEST_SCALED)
    # Divide train/test General
    df_feat = df_ua_fs[[x for x in df_ua_fs.columns if x.startswith('f_')]+['cellID']].set_index('cellID')
    df_target = df_ua_fs[[x for x in df_ua_fs.columns if x.startswith('t_')]+['cellID']].set_index('cellID')
    df_X_train, df_X_test, df_y_train, df_y_test = train_test_split(df_feat, df_target, test_size=0.2, random_state=42, stratify=df_target)

    df_X_train.dropna(inplace=True)    
    df_X_test.dropna(inplace=True)    
    #     df_X_test.fillna(0, inplace=True)
    
    # save datasets
    df_train = df_X_train.merge(df_y_train, left_index=True, right_index=True)
    df_test = df_X_test.merge(df_y_test, left_index=True, right_index=True)    
    df_train.to_csv(OUTPUT_TRAIN, index_label="cellID", sep="\t",float_format='%.6f')
    df_test.to_csv(OUTPUT_TEST, index_label="cellID", sep="\t",float_format='%.6f')
    

    # Create scaled version of train and test
    X_scaler = StandardScaler()
    df_X_train_scaled = X_scaler.fit_transform(df_X_train)
    df_X_train_scaled = pd.DataFrame(df_X_train_scaled, index=df_X_train.index, columns=df_X_train.columns)
    df_X_test_scaled = X_scaler.transform(df_X_test)
    df_X_test_scaled = pd.DataFrame(df_X_test_scaled, index=df_X_test.index, columns=df_X_test.columns)

    df_train_scaled = df_X_train_scaled.merge(df_y_train,left_index=True, right_index=True)
    df_test_scaled = df_X_test_scaled.merge(df_y_test,left_index=True, right_index=True)

    df_train_scaled.to_csv(OUTPUT_TRAIN_SCALED, index_label="cellID", sep="\t",float_format='%.6f')
    df_test_scaled.to_csv(OUTPUT_TEST_SCALED, index_label="cellID", sep="\t",float_format='%.6f')

    
    return df_train,df_test

### Run Experiments

In [21]:
def printEvalutationMetrics(df_y_test,y_pred):
    print( metrics.classification_report(df_y_test.values, y_pred))
    print( "ACCURACY: {}".format(metrics.accuracy_score(df_y_test.values, y_pred)))
    print("F1 SCORE: {}".format(metrics.f1_score(df_y_test.values, y_pred, average='macro')))
    
def runExperiment(df_train, df_test):

    dfs = []
    dim = 200
    df = {} #{"area": boro, "cell": dim}

    suffix_train = "General"
    suffix_test = "General"
    
    df_y_train = df_train['t_predominant']
    df_y_test = df_test['t_predominant']
    
    # Baseline
    df_train['t_predominant'].value_counts().max()
    y_pred = [df_train['t_predominant'].value_counts().idxmax()] * len(df_y_test)
    
    print( "*****************************"+CITY_NAME+"  "+SIZE+"*********************************")
    
    print("****** BASELINE ******")
    # Print Metrics
    printEvalutationMetrics(df_y_test,y_pred)
    df['model'] = "baseline"

    df['accuracy'] = metrics.accuracy_score(df_y_test.values, y_pred) #metrics.accuracy_score(df_y_test.values, y_pred)
    df['f1-score'] = metrics.f1_score(df_y_test.values, y_pred,average='macro') #metrics.accuracy_score(df_y_test.values, y_pred)
    df['precision'] = metrics.precision_score(df_y_test.values, y_pred,average='macro') #metrics.accuracy_score(df_y_test.values, y_pred)
    df['recall'] = metrics.recall_score(df_y_test.values, y_pred,average='macro') #metrics.accuracy_score(df_y_test.values, y_pred)

    dfs.append(df)
    print( "**********************")

    # # xgboost Classifier
    df = {}
    print( "****** XGBOOST ******")
    df_X_train = df_train[[c for c in df_train.columns if c.startswith('f_')]]
    df_X_test = df_test[[c for c in df_test.columns if c.startswith('f_')]]
    
    clf = xgboost.XGBClassifier()#colsample_bytree=0.8, scale_pos_weight=1, learning_rate=0.1, min_child_weight=5,n_estimators=177, subsample=0.8, max_depth=3, gamma=0)
    clf.fit(df_X_train.as_matrix(), df_y_train.values)
    y_pred = clf.predict(df_X_test.as_matrix())
    # Print Metrics
    printEvalutationMetrics(df_y_test,y_pred)
    df['model'] = 'GBT'
    df['accuracy'] = metrics.accuracy_score(df_y_test.values, y_pred) #metrics.accuracy_score(df_y_test.values, y_pred)
    df['f1_score'] = metrics.f1_score(df_y_test.values, y_pred, average='macro') 
    print('********* CONFUSION MATRIX *******************')
    print(confusion_matrix(df_y_test.values, y_pred))
    
    print("********************************************************************************")

In [22]:
# iterate over city name
for CITY_NAME in CITIES:

    BASE_DIR_CITY =  os.path.join(BASE_DIR_DATA, CITY_NAME)    
    BASE_DIR_TESSELLATION = os.path.join(BASE_DIR_CITY, "tessellation")
    
    #  iterate over grid size
    for INPUT_TESSELLATION in os.listdir(BASE_DIR_TESSELLATION):
        
        INPUT_DIR = os.path.join(BASE_DIR_TESSELLATION,INPUT_TESSELLATION)
        SIZE = INPUT_DIR.split('-')[-1].split('.')[0]
        
        # merge FS + UA
        df_ua_fs = mergeUA_POI(CITY_NAME, SIZE)
            
        # divide Train/Test 
        df_train,df_test =split_train_test(df_ua_fs, CITY_NAME, SIZE)

        # run experiment
        runExperiment(df_train, df_test)

/home/public/geotk/data/barcelona/train/barcelona_fs_250.csv /home/public/geotk/data/barcelona/test/barcelona_fs_250.csv
/home/public/geotk/data/barcelona/train/barcelona_fs_250_scaled_.csv /home/public/geotk/data/barcelona/test/barcelona_fs_250_scaled_.csv
*****************************barcelona  250*********************************
****** BASELINE ******
             precision    recall  f1-score   support

       Agri       0.00      0.00      0.00         2
    Forests       0.00      0.00      0.00        11
Green_Urban       0.00      0.00      0.00        23
         HD       0.54      1.00      0.70       216
 Industrial       0.00      0.00      0.00        72
         LD       0.00      0.00      0.00        12
         MD       0.00      0.00      0.00        13
     Sports       0.00      0.00      0.00        14
  Transport       0.00      0.00      0.00        36

avg / total       0.29      0.54      0.38       399

ACCURACY: 0.5413533834586466
F1 SCORE: 0.078048780487804

GeoL/poi_number.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import matplotlib.pyplot as plt
GeoL/poi_number.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ferretti/trento/lib/python3.4/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ferretti/trento/lib/python3.4/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, 

             precision    recall  f1-score   support

       Agri       0.00      0.00      0.00         2
    Forests       0.23      0.27      0.25        11
Green_Urban       0.20      0.09      0.12        23
         HD       0.75      0.92      0.83       216
 Industrial       0.42      0.47      0.44        72
         LD       0.00      0.00      0.00        12
         MD       0.00      0.00      0.00        13
     Sports       0.29      0.14      0.19        14
  Transport       0.45      0.25      0.32        36

avg / total       0.55      0.62      0.58       399

ACCURACY: 0.6215538847117794
F1 SCORE: 0.23917348084014753
********* CONFUSION MATRIX *******************
[[  0   1   0   0   1   0   0   0   0]
 [  0   3   1   1   4   0   0   0   2]
 [  0   1   2  11   5   0   0   3   1]
 [  0   0   3 198  14   0   0   1   0]
 [  0   3   1  27  34   1   1   1   4]
 [  0   2   1   2   5   0   0   0   2]
 [  0   1   0   6   4   0   0   0   2]
 [  0   0   2   6   3   1   0   2  

GeoL/poi_number.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import matplotlib.pyplot as plt
GeoL/poi_number.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


*****************************barcelona  50*********************************
****** BASELINE ******
             precision    recall  f1-score   support

       Agri       0.00      0.00      0.00        13
    Forests       0.00      0.00      0.00         9
Green_Urban       0.00      0.00      0.00        91
         HD       0.68      1.00      0.81      2640
 Industrial       0.00      0.00      0.00       421
         LD       0.00      0.00      0.00        38
         MD       0.00      0.00      0.00        75
     Sports       0.00      0.00      0.00       107
  Transport       0.00      0.00      0.00       467

avg / total       0.47      0.68      0.56      3861

ACCURACY: 0.6837606837606838
F1 SCORE: 0.09024252679075014
**********************
****** XGBOOST ******


...

GeoL/poi_number.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import matplotlib.pyplot as plt
GeoL/poi_number.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


*****************************lisbon  50*********************************
****** BASELINE ******
             precision    recall  f1-score   support

       Agri       0.00      0.00      0.00         6
    Forests       0.00      0.00      0.00         4
Green_Urban       0.00      0.00      0.00        83
         HD       0.56      1.00      0.72      1108
 Industrial       0.00      0.00      0.00       367
         LD       0.00      0.00      0.00        10
         MD       0.00      0.00      0.00        48
     Sports       0.00      0.00      0.00        53
  Transport       0.00      0.00      0.00       302
      Water       0.00      0.00      0.00         1

avg / total       0.31      0.56      0.40      1982

ACCURACY: 0.5590312815338042
F1 SCORE: 0.07171521035598705
**********************
****** XGBOOST ******


/home/ferretti/trento/lib/python3.4/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

       Agri       0.00      0.00      0.00         6
    Forests       0.00      0.00      0.00         4
Green_Urban       0.00      0.00      0.00        83
         HD       0.57      0.96      0.72      1108
 Industrial       0.53      0.14      0.22       367
         LD       0.00      0.00      0.00        10
         MD       0.00      0.00      0.00        48
     Sports       0.00      0.00      0.00        53
  Transport       0.42      0.04      0.07       302
      Water       0.00      0.00      0.00         1

avg / total       0.48      0.57      0.45      1982

ACCURACY: 0.5696266397578204
F1 SCORE: 0.10063782419594325
********* CONFUSION MATRIX *******************
[[   0    0    0    6    0    0    0    0    0    0]
 [   0    0    0    4    0    0    0    0    0    0]
 [   0    0    0   80    3    0    0    0    0    0]
 [   0    0    0 1067   31    0    0    0   10    0]
 [   0    0    0  312   51    0    0    0  

/home/ferretti/trento/lib/python3.4/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ferretti/trento/lib/python3.4/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ferretti/trento/lib/python3.4/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
